# Linear regression applied to Option Pricing

This code applies linear regression to option pricing under the Black–Scholes model (BSM) - a widely used differential equation model to price option contracts [[Ref](https://www.investopedia.com/terms/b/blackscholes.asp)].

We make use of `BS_Fun.py` for generating data and some other functions.

## Generating the data

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')
## IMPORTANT: replace with the path to your local drive
os.chdir('drive/My Drive/Teaching/LT_2021_2022/ST456/Week03')

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
# helper module for generating the dataset
from BS_Fun import *

In [ ]:
'''
  See lines 8 to 15 on BS_Fun.py for further details.
  The bs_call function takes in:
    the current price of an asset (S)
    the strike price (K)
    the maturity (T),
    the interest rate (r),
    and volatility (sigma)

  In the below example, we take S=10, K=15, T=1, r=0, sigma=1
'''
bs_call(10,*[15,1,0,1])

In [ ]:
'''
In the below example, we take a variety of S, and fix the same K, T, r, sigma as previous.
This enables us to further plot the price of the options against various strike price, ceteris paribus.
'''
s = np.arange(1,40,0.1)
trial = BS_Options_Pricing_S(s,[15,1,0,1])
trial.make_calls()
trial.make_puts()

In [ ]:
data=df({'Stock price':s,
    'Calls':trial.calls,
    'Puts':trial.puts}
       )
data.plot('Stock price',figsize=(15,5),legend=True,grid=True)

## Non-linear approximation

### An example of $\mathbb{R} \to \mathbb{R}$

In [ ]:
def generate_sample_no_noise(S=20,K=np.arange(10,30,0.1),
                             T=1,r=0,true_sigma=1):
    '''
    We receive data over different calls, and would like to predict further on the other strike prices
    Suppose we do not have access to sigma, so we use neural networks to make the predictions.
    We first assume that we have no noise in our data :)
    '''
    trial=BS_Options_Pricing_K(K,[S,T,r,true_sigma])
    trial.make_calls()
    data =df({'Strike price':K, 'Call price':trial.calls})
    return data

In [ ]:
#(x_1,y_1),...,(x_n,y_n)
data = generate_sample_no_noise()

In [ ]:
len(data)

In [ ]:
data.head()

In [ ]:
data.plot.scatter('Strike price','Call price')
# sns.lmplot(x='Strike price',y='Call price',data=data,fit_reg=True)

## Model definition 


In [ ]:
# importing necessary libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# print(tf.__version__)

In [ ]:
# if you want the results to be reproducible.
seed_value=0
tf.random.set_seed(seed_value)
np.random.seed(seed_value)

### Data split and normalization

In [ ]:
# splitting into features and label (target)
feature = data['Strike price']
target  = data['Call price']

# feature normalization layer (https://www.tensorflow.org/api_docs/python/tf/keras/layers/Normalization)
# shift and scale inputs into a distribution centered around 0 with standard deviation 1.
# The mean and variance values for the layer must be either supplied on construction or learned via adapt(). 
# adapt() will compute the mean and variance of the data and store them as the layer's weights. 
# adapt() should be called before fit(), evaluate(), or predict().
feature_normaliser = layers.Normalization(input_shape=[1,], axis=None, name='normaliser')
feature_normaliser.adapt(feature)

### Model definition

We will be using a Tensorflow [regression model](https://www.tensorflow.org/tutorials/keras/regression).


In [ ]:
# sequential model
linear_regression = keras.Sequential(name='First_model')
# feature normalization applied to the input features
linear_regression.add(feature_normaliser)
# linear model
linear_regression.add(layers.Dense(units=1, name='Dense'))

# we can get a summary of our model
linear_regression.summary()

# and compile the model with "standard" parameters: SGD, learning rate and loss function
linear_regression.compile(
    optimizer=tf.optimizers.SGD(learning_rate=0.1),
    loss='mean_squared_error')

Question to discuss in group [3 mins]: why are there 5 parameters? 
Write down the model and illustrate these 5 parameters.

---

### Training the model

In [ ]:
# we use %%time to get some execution time statistics
%%time
## history is a dictionary capturing training data
history = linear_regression.fit(
    feature,
    target,
    epochs=100,
    # suppress logging.
    verbose=0,
    # calculate validation results on 0% of the training data.
    validation_split = 0,
    validation_data = None)

#Reference: https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit

plt.plot(history.history['loss'], label='loss')
plt.grid(True)
plt.xlabel('Epoch')
plt.ylabel('Error')
plt.legend()
plt.show()

In [ ]:
# model's final weights
linear_regression.get_weights()

### Predicting targets

In [ ]:
y_pred = linear_regression.predict(feature)

In [ ]:
plt.plot(y_pred, label='ypred')
plt.plot(target, label='target')
plt.legend()
plt.grid(True)
plt.xlabel('Number of samples')
plt.ylabel('Call price')
plt.show()

---

**Exercise:** evaluate the model using three different methods:

	* `keras.metrics`
	* `model.evaluate`
	* `history.history`

In [ ]:
# For students practices

### Non-linear regression

In [ ]:
Model1 = keras.Sequential(name='One_layer_model')
Model1.add(feature_normaliser)
Model1.add(layers.Dense(1,activation='sigmoid',name='sigmoid'))
Model1.add(layers.Dense(1,name='dense_after_sigmoid'))

Model1.summary()

Model1.compile(
    optimizer=tf.optimizers.SGD(learning_rate=0.1),
    loss='mean_squared_error')

Exercise and Question to discuss in group [5 mins]: Write down the formula and disuss what is expected for the result.

In [ ]:
history = Model1.fit(
    feature,
    target,
    epochs=100,
    verbose=0,
    validation_split = 0,
    validation_data = None)

plt.plot(history.history['loss'])
plt.grid(True)
plt.show()

In [ ]:
Model1.get_weights()

In [ ]:
y_pred=Model1.predict(feature)

In [ ]:
plt.plot(y_pred)
plt.plot(target)

In [ ]:
y_pred = y_pred.reshape(target.shape)
keras.metrics.mean_squared_error(target,y_pred).numpy()

In [ ]:
Model1.evaluate(feature,target,verbose=0)

### Non-linear regression (second model)

In [ ]:
Model2 = keras.Sequential(name='Two_layers_model')
Model2.add(feature_normaliser)
Model2.add(layers.Dense(2,activation='sigmoid',name='sigmoid0'))
Model2.add(layers.Dense(2,name='linear_2X2'))
Model2.add(layers.Dense(4,activation='sigmoid',name='sigmoid1'))
Model2.add(layers.Dense(1,name='linear_4X1'))

Model2.summary()

Model2.compile(
    optimizer=tf.optimizers.SGD(learning_rate=0.1),
    loss='mean_squared_error')

In [ ]:
history = Model2.fit(
    feature,
    target,
    epochs=100,
    verbose=0,
    validation_split = 0,
    validation_data = None)

plt.plot(history.history['loss'])
plt.grid(True)
plt.show()

In [ ]:
y_pred=Model2.predict(feature)

plt.plot(y_pred)
plt.plot(target)

In [ ]:
Model2.evaluate(feature,target,verbose=0)

### Summary of achievements

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(linear_regression.predict(feature), label='Linear Model')
plt.plot(Model1.predict(feature), label='Model1')
plt.plot(Model2.predict(feature), label='Model2')
plt.plot(target, label='Actual_data')
plt.legend()
plt.grid(True)

## Validation loss

We now look at the validation step, more precisely the validation loss.

We use the same models from the previous section, training over 100 epochs. The difference is that we make use of the `validation_data` parameter to specify a portion of the dataset to be used for validation purposes (i.e., we don't train the model over this portion).

In [ ]:
seed_value = 0
tf.random.set_seed(seed_value)
np.random.seed(seed_value)

`data_validiation` will be the data samples used for validation, whereas `data_train` is the training sample.

In [ ]:
data = generate_sample_no_noise(K=np.arange(1,40,0.01))
indices = np.random.randint(low=0, high=data.shape[0], size=int(data.shape[0]*0.3))
data_validiation = data.loc[indices]
data_train = data.drop(indices)

In [ ]:
feature_train = data_train['Strike price']
target_train  = data_train['Call price']
feature_normaliser = layers.Normalization(input_shape=[1,], axis=None,name='normaliser')
feature_normaliser.adapt(feature_train)

### First model using validation data

In [ ]:
linear_regression = keras.Sequential(name='First_model')
linear_regression.add(feature_normaliser)
linear_regression.add(layers.Dense(units=1, name='dense'))

linear_regression.compile(
    optimizer=tf.optimizers.SGD(learning_rate=0.1),
    loss='mean_squared_error')

history0 = linear_regression.fit(
    feature_train,
    target_train,
    epochs=100,
    verbose=0,
    validation_data = (data_validiation['Strike price'],
                      data_validiation['Call price']))

plt.plot(history0.history['loss'], label='training loss')
plt.plot(history0.history['val_loss'], label='validation loss')
plt.legend()
plt.grid(True)
plt.show()

### Second and third models using validation data

In [ ]:
Model1 = keras.Sequential(name='One_layer_model')
Model1.add(feature_normaliser)
Model1.add(layers.Dense(1,activation='sigmoid',name='sigmoid'))
Model1.add(layers.Dense(1,name='dense_after_sigmoid'))

Model1.compile(
    optimizer=tf.optimizers.SGD(learning_rate=0.1),
    loss='mean_squared_error')

Model2 = keras.Sequential(name='Two_layers_model')
Model2.add(feature_normaliser)
Model2.add(layers.Dense(2,activation='sigmoid',name='sigmoid0'))
Model2.add(layers.Dense(2,name='linear_2X2'))
Model2.add(layers.Dense(4,activation='sigmoid',name='sigmoid1'))
Model2.add(layers.Dense(1,name='linear_4X1'))

Model2.compile(
    optimizer=tf.optimizers.SGD(learning_rate=0.1),
    loss='mean_squared_error')

history1 = Model1.fit(
    feature_train,
    target_train,
    epochs=100,
    verbose=0,
    validation_data = (data_validiation['Strike price'],
                      data_validiation['Call price']))

history2 = Model2.fit(
    feature_train,
    target_train,
    epochs=100,
    verbose=0,
    validation_data = (data_validiation['Strike price'],
                      data_validiation['Call price']))
plt.figure()
plt.plot(history1.history['loss'], label='Model 1 training loss',color='darkviolet')
plt.plot(history1.history['val_loss'], label='Model 1 validation loss',color='violet')
plt.legend()
plt.grid(True)

plt.figure()
plt.plot(history2.history['loss'], label='Model 2 training loss',color='darkgreen')
plt.plot(history2.history['val_loss'], label='Model 2 validation loss',color='lime')
plt.legend()
plt.grid(True)
plt.show()

### Comparing models

In [ ]:
plt.plot(history0.history['loss'], label='Linear model training loss')
plt.plot(history1.history['loss'], label='Model 1 training loss',color='darkviolet')
plt.plot(history2.history['loss'], label='Model 2 training loss',color='darkgreen')
plt.ylim([0, 2])
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
plt.plot(history0.history['val_loss'], label='Linear model validation loss',color='royalblue')
plt.plot(history1.history['val_loss'], label='Model 1 validation loss',color='violet')
plt.plot(history2.history['val_loss'], label='Model 2 validation loss',color='lime')
plt.ylim([0,2])
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
data_validiation.sort_index(ascending=True,inplace=True)

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(linear_regression.predict(data_validiation['Strike price']),label='Linear Model')
plt.plot(Model1.predict(data_validiation['Strike price']),label='Model1')
plt.plot(Model2.predict(data_validiation['Strike price']),label='Model2')
plt.plot(data_validiation['Call price'].values,label='data')
plt.legend()
plt.grid(True)

## Validation / testing in unseen region

In [ ]:
data_test = generate_sample_no_noise(K=np.arange(40,50,0.01))
#data_test

**Exercise:** What would happen now to testing data in unseen region?

`model.predict(data_test['Strike price'])`